In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=6feac23d20e139b7492384ea835b42d6e98eec34c8b9a5094dec0b88b0d87616
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=8690ea6748a7c77317afab8c0b5c8a4f96cca9bc86a0a87b6ca6fd4f4fd982e2
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras 
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


def setup_dataset(text):
  import shutil
  import os
  %cd /content/gdrive/My Drive/Kaggle/
  strings = text.split(' ')[-1]
  folder = strings.split('/')[1]
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/"+folder
  print(strings)
  print(folder)
  !mkdir $folder
  shutil.copy2("kaggle.json","./"+folder+"/kaggle.json")
  %cd $folder
  !kaggle datasets download -d $strings
  
setup_dataset("kaggle datasets download -d rashikrahmanpritom/heart-attack-analysis-prediction-dataset")

Mounted at /content/gdrive
/content/gdrive/My Drive/Kaggle
rashikrahmanpritom/heart-attack-analysis-prediction-dataset
heart-attack-analysis-prediction-dataset
mkdir: cannot create directory ‘heart-attack-analysis-prediction-dataset’: File exists
/content/gdrive/My Drive/Kaggle/heart-attack-analysis-prediction-dataset
heart-attack-analysis-prediction-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
%ls

heart-attack-analysis-prediction-dataset.zip  o2Saturation.csv
heart.csv                                     untitled_project/
kaggle.json


In [5]:
!unzip heart-attack-analysis-prediction-dataset.zip

Archive:  heart-attack-analysis-prediction-dataset.zip
replace heart.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace o2Saturation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [6]:
df = pd.read_csv("heart.csv")

In [7]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [8]:
x.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [9]:
from sklearn.model_selection import train_test_split


In [10]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.15
                                                 )

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
ss = StandardScaler()

In [13]:
x_train = ss.fit_transform(x_train)

In [14]:
x_test = ss.fit_transform(x_test)

In [15]:
x_train.shape[1]

13

##Model Creation with hyperparameter optimisation

In [37]:
!rm -r heart_model

In [38]:
def build_model(hp):
  model = keras.Sequential()  #Model Definition

  for i in range(hp.Int('num_layers',2,20)):       #Select number of hidden layers from 2 to 20
    model.add(layers.Dense(units = hp.Int('units_'+str(i),min_value = 4,max_value = 516,step = 16),
                           activation = "relu"
                          )
              )
  model.add(layers.Dense(1,activation = "sigmoid"))
  model.compile(
      optimizer = keras.optimizers.Adam(
          hp.Choice('learning_rate',[1e-1,1e-2,1e-3,1e-4]),
      ),
      loss = "binary_crossentropy",
      metrics = ["binary_accuracy"]
  )
  return model

In [39]:
tuner = RandomSearch(
    build_model,
    objective = "val_binary_accuracy",
    max_trials = 5,
    executions_per_trial = 3,
    directory='heart_model',
    project_name='heart_classification'
)

In [40]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 516, 'step': 16, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 516, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01, 0.001, 0.0001], 'ordered': True}


In [41]:
tuner.search(x_train,y_train,epochs =50,validation_data = (x_test,y_test))

Trial 5 Complete [00h 00m 25s]
val_binary_accuracy: 0.7898550629615784

Best val_binary_accuracy So Far: 0.8478260636329651
Total elapsed time: 00h 01m 39s
INFO:tensorflow:Oracle triggered exit


In [42]:
tuner.results_summary()

Results summary
Results in heart_model/heart_classification
Showing 10 best trials
Objective(name='val_binary_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 372
units_1: 356
learning_rate: 0.01
Score: 0.8478260636329651
Trial summary
Hyperparameters:
num_layers: 5
units_0: 180
units_1: 212
learning_rate: 0.001
units_2: 4
units_3: 4
units_4: 4
Score: 0.8405796885490417
Trial summary
Hyperparameters:
num_layers: 6
units_0: 260
units_1: 132
learning_rate: 0.0001
units_2: 436
units_3: 116
units_4: 68
units_5: 436
units_6: 84
units_7: 164
units_8: 100
units_9: 340
Score: 0.8260869383811951
Trial summary
Hyperparameters:
num_layers: 5
units_0: 20
units_1: 100
learning_rate: 0.0001
units_2: 180
units_3: 484
units_4: 468
units_5: 164
units_6: 100
units_7: 308
units_8: 324
units_9: 372
Score: 0.7898550629615784
Trial summary
Hyperparameters:
num_layers: 10
units_0: 420
units_1: 132
learning_rate: 0.1
units_2: 116
units_3: 100
units_4: 484
units_5: 4
units_6: 4

##Using Best model from above to create the best performing NN

In [43]:
"""
Hyperparameters:
num_layers: 2
units_0: 372
units_1: 356
learning_rate: 0.01
"""

'\nHyperparameters:\nnum_layers: 2\nunits_0: 372\nunits_1: 356\nlearning_rate: 0.01\n'

In [84]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

In [85]:
best_model = keras.Sequential()

In [86]:
best_model.add(Dense(372,activation="relu",input_dim = 13))

In [87]:
best_model.add(Dense(356,activation="relu"))

In [88]:
best_model.add(Dense(1,activation="sigmoid"))

In [89]:
opt = keras.optimizers.Adam(learning_rate=0.01)

In [90]:
best_model.compile(optimizer=opt,loss ="binary_crossentropy",metrics="binary_accuracy")

In [91]:
best_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 372)               5208      
_________________________________________________________________
dense_19 (Dense)             (None, 356)               132788    
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 357       
Total params: 138,353
Trainable params: 138,353
Non-trainable params: 0
_________________________________________________________________


In [92]:
model_history = best_model.fit(x_train,y_train,validation_split=0.2,batch_size=64,epochs=100,workers = -1)

Epoch 1/100
4/4 [==============================] - 1s 76ms/step - loss: 0.5707 - binary_accuracy: 0.6309 - val_loss: 0.6503 - val_binary_accuracy: 0.8077
Epoch 2/100
4/4 [==============================] - 0s 21ms/step - loss: 0.3288 - binary_accuracy: 0.8619 - val_loss: 0.5351 - val_binary_accuracy: 0.8269
Epoch 3/100
4/4 [==============================] - 0s 22ms/step - loss: 0.2907 - binary_accuracy: 0.8786 - val_loss: 0.5657 - val_binary_accuracy: 0.7885
Epoch 4/100
4/4 [==============================] - 0s 25ms/step - loss: 0.2758 - binary_accuracy: 0.8710 - val_loss: 0.5416 - val_binary_accuracy: 0.8269
Epoch 5/100
4/4 [==============================] - 0s 22ms/step - loss: 0.2002 - binary_accuracy: 0.9153 - val_loss: 0.5556 - val_binary_accuracy: 0.8269
Epoch 6/100
4/4 [==============================] - 0s 22ms/step - loss: 0.2037 - binary_accuracy: 0.9145 - val_loss: 0.5749 - val_binary_accuracy: 0.8462
Epoch 7/100
4/4 [==============================] - 0s 23ms/step - loss: 0.15

In [93]:
y_pred = best_model.predict(x_test)

In [94]:
y_pred = y_pred>0.5

In [95]:
from sklearn.metrics import confusion_matrix

In [96]:
print(confusion_matrix(y_test,y_pred))

[[16  5]
 [ 5 20]]


In [97]:
from sklearn.metrics import accuracy_score


In [98]:
score = accuracy_score(y_test,y_pred)

In [99]:
print(score)

0.782608695652174
